In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from collections import deque

#read frames
cap = cv2.VideoCapture(0)

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
colorIndex = 0

#use deques to store separate colors in separate arrays
bpoints = [deque()]
gpoints = [deque()]
rpoints = [deque()]
ypoints = [deque()]

blue_index = 0
green_index = 0
red_index = 0
yellow_index = 0

# using lower and upper boundaries due to hsv color space.
#we now use color blue in order to use any other color just adjust the folloing code with its boundaries 
Lower = np.array([100, 60, 60])
Upper = np.array([140, 255, 255])

kernel = np.ones((5,5),np.uint8)


#cv2.circle(frame, center_coordinates, radius, color, thickness)
#cv2.rectangle(img, start_point, end_point, color, thickness)
paintWindow = np.zeros((471,636,3)) + 255
paintWindow = cv2.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2)
paintWindow = cv2.rectangle(paintWindow, (160,1), (255,65), colors[0], -1)
paintWindow = cv2.rectangle(paintWindow, (275,1), (370,65), colors[1], -1)
paintWindow = cv2.rectangle(paintWindow, (390,1), (485,65), colors[2], -1)
paintWindow = cv2.rectangle(paintWindow, (505,1), (600,65), colors[3], -1)

cv2.putText(paintWindow, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)


while True:
   
    ret, frame = cap.read()
    #ret is a boolean variable that returns true if the frame is available.
    #frame is an image array vector captured based on the default frames per second defined explicitly or implicitly
    frame = cv2.flip(frame, 1)
    #frame = cv2.flip(frame, 0)#0 is for flip vertically, and 1 for flip horizontally
    # Convert the frame BGR to HSV color space

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #putting buttons and text together for ease at chosing the color
   
    frame = cv2.rectangle(frame, (40,1), (140,65), (122,122,122), -1)
    frame = cv2.rectangle(frame, (160,1), (255,65), colors[0], -1)
    frame = cv2.rectangle(frame, (275,1), (370,65), colors[1], -1)
    frame = cv2.rectangle(frame, (390,1), (485,65), colors[2], -1)
    frame = cv2.rectangle(frame, (505,1), (600,65), colors[3], -1)
    cv2.putText(frame, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)


   # Create a binary segmented mask of green color
#returns a binary segmented mask containing a white region where the color is detected and the rest of the region as black.

    Mask = cv2.inRange(hsv, Lower, Upper)
    Mask = cv2.erode(Mask, kernel, iterations=1)
    Mask = cv2.morphologyEx(Mask, cv2.MORPH_OPEN, kernel)
    # Add some dialation to increase segmented  white region
    Mask = cv2.dilate(Mask, kernel, iterations=1)

    # Find all the contours of the segmented mask

    contours,h = cv2.findContours(Mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    center = None
    #finds all the continuous points along the boundary having the same intensity or color
    
    # Checking if any contour is detected then run the following statements

    if len(contours) > 0:

        cnt = sorted(contours, key = cv2.contourArea, reverse = True)[0]        
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
            # Drawing a circle in the center of the contour area

        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
        M = cv2.moments(cnt)
            # Find center point of the contour

        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
        # Selecting the color for drawing in the canvas

        if center[1] <= 65:
            if 40 <= center[0] <= 140: #clear
                bpoints = [deque(maxlen=512)]
                gpoints = [deque(maxlen=512)]
                rpoints = [deque(maxlen=512)]
                ypoints = [deque(maxlen=512)]

                blue_index = 0
                green_index = 0
                red_index = 0
                yellow_index = 0

                paintWindow[67:,:,:] = 255
            elif 160 <= center[0] <= 255: #blue
                    colorIndex = 0 
            elif 275 <= center[0] <= 370:#green
                    colorIndex = 1 
            elif 390 <= center[0] <= 485:#red
                    colorIndex = 2 
            elif 505 <= center[0] <= 600:#yellow
                    colorIndex = 3 
        else :
            if colorIndex == 0:
                bpoints[blue_index].appendleft(center)
            elif colorIndex == 1:
                gpoints[green_index].appendleft(center)
            elif colorIndex == 2:
                rpoints[red_index].appendleft(center)
            elif colorIndex == 3:
                ypoints[yellow_index].appendleft(center)
    else:    # Append the next deque when no contours are detected

        bpoints.append(deque(maxlen=512))
        blue_index += 1
        gpoints.append(deque(maxlen=512))
        green_index += 1
        rpoints.append(deque(maxlen=512))
        red_index += 1
        ypoints.append(deque(maxlen=512))
        yellow_index += 1

# Draw line
    points = [bpoints, gpoints, rpoints, ypoints]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

   
    cv2.imshow("Tracking", frame)
    cv2.imshow("Paint", paintWindow)
    cv2.imshow("mask",Mask)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

